In [ ]:
%pip install boto3
%pip install botocore


In [ ]:
import boto3
import botocore
import uuid
import json
import time

In [ ]:
region = "us-west-2"

bedrock_agent_client = boto3.client('bedrock-agent', region_name=region)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name=region)
iam_client = boto3.client('iam')

### 1. Create the role that the agent will assume

In [ ]:
role_name = "MyBedrockAgentRole"

# Define the trust policy for Bedrock service
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Create the IAM role with the trust policy
try:
    print(f"Creating IAM role: {role_name}...")
    response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description="IAM role for Amazon Bedrock agent"
    )
    
    role_arn = response['Role']['Arn']
    print(f"IAM role created successfully with ARN: {role_arn}")
    
except iam_client.exceptions.EntityAlreadyExistsException:
    print(f"IAM role {role_name} already exists. Getting its ARN...")
    response = iam_client.get_role(RoleName=role_name)
    role_arn = response['Role']['Arn']
    print(f"Using existing role ARN: {role_arn}")

### 2. Create the necessary policies and attach them to the role

In [ ]:
policy_name = f"{role_name}Policy" # the policy to invoke Bedrock

# first check if the policy exists, and delete it
is_truncated = True
list_policies_response = iam_client.list_policies(Scope='Local')
policies = list_policies_response['Policies']
while is_truncated:
    print(f'Policies retrieved: {len(policies)}')
    for policy in policies:
        if policy['PolicyName'] == policy_name:
            print(f'Policy already exists with ARN {policy["Arn"]}\nwill detach from the role and then delete it ...')
            iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy['Arn'])
            iam_client.delete_policy(PolicyArn=policy['Arn'])
            break
    is_truncated = list_policies_response['IsTruncated']
    if is_truncated:
        policies = iam_client.list_policies(Marker=list_policies_response['Marker'],Scope='Local')['Policies']

bedrock_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream"
            ],
            "Resource": "*"
        }
    ]
}

# Create the policy

print(f"Creating policy: {policy_name}...")
policy_response = iam_client.create_policy(
    PolicyName=policy_name,
    PolicyDocument=json.dumps(bedrock_policy_document),
    Description="Policy for Amazon Bedrock agent permissions"
)

policy_arn = policy_response['Policy']['Arn']
print(f"Policy created successfully with ARN: {policy_arn}")

# now attach the policy to the role
print(f"Attaching policy to role...")
iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

### 3. Create the agent

In [ ]:
agent_name = 'booking-agent'
agent_description = 'Agent in charge of a restaurants table bookings'
agent_instruction = """
You are a restaurant agent, helping clients retrieve information from their booking,
create a new booking or delete an existing booking
"""
agent_foundation_model = 'anthropic.claude-3-5-sonnet-20241022-v2:0'

# check if the agent exists and delete it if already there
agent_id = None
response = bedrock_agent_client.list_agents()
next_token = response.get('nextToken')
while True:
    for agent in response['agentSummaries']:
        if agent['agentName'] == agent_name:
            print("Agent already exists, will delete it ...")
            agent_id = agent['agentId']
            bedrock_agent_client.delete_agent(agentId=agent_id)
            try:
                while bedrock_agent_client.get_agent(agentId=agent_id):
                    print("Agent is being deleted ...")
                    time.sleep(5)
            except bedrock_agent_client.exceptions.ResourceNotFoundException:
                    print("Agent has been deleted")
            next_token = None # break the outer loop ...
            break
    if next_token:
        response = bedrock_agent_client.list_agents(nextToken=next_token)
        next_token = response.get('nextToken')
    else:
        break

# now cretae the agent
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=role_arn,
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

while bedrock_agent_client.get_agent(agentId=agent_id)['agent']["agentStatus"] == 'CREATING':
    print("Agent is being created ...")
    time.sleep(5)

### 4. Create the Dynamo DB where the Agent will store the bookings

In [ ]:
dynamodb = boto3.client("dynamodb",region_name=region)
table_name = "restaurant_bookings"

# check if the table exists; if it doesn't, then create the table
try:
    response = dynamodb.describe_table(TableName=table_name)
    print(f'Table <{table_name}> already exists ...')
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'ResourceNotFoundException':
        print(f'Table <{table_name}> does not exist, creating the table ...')
        table = dynamodb.create_table(
            TableName=table_name,
            KeySchema=[
                {
                    'AttributeName': 'booking_id',
                    'KeyType': 'HASH'
                }
            ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'booking_id',
                    'AttributeType': 'S'
                }
            ],
            BillingMode='PAY_PER_REQUEST'  # Use on-demand capacity mode
        )


        # Wait until the table exists.
        dynamodb.get_waiter("table_exists").wait(TableName=table_name)

        print(f'Table <{table_name}> has been created')


In [ ]:
# create an iam role that a lambda fuction can assume to be able to get, put, delete items in the dynamo db table
# if the role already exists, it will be used, otherwise it will be created
iam_client = boto3.client('iam')
    
assume_role_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow", 
        "Principal": {"Service": "lambda.amazonaws.com"},
        
        "Action": "sts:AssumeRole"
    }]
}

dynamo_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Action": [
            "dynamodb:GetItem",
            "dynamodb:PutItem", 
            "dynamodb:DeleteItem"
        ],
        
        "Resource": "*"
    }]
}

lambda_role_name = 'LambdaDynamoDBRole'
try:
    response = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy)
    )
    lambda_role_arn = response['Role']['Arn']
    print(f"IAM role created successfully with ARN: {role_arn}")

    policy = iam_client.create_policy(
        PolicyName='LambdaDynamoDBPolicy',
        PolicyDocument=json.dumps(dynamo_policy)
    )

    iam_client.attach_role_policy(
        RoleName='LambdaDynamoDBRole',
        PolicyArn=policy['Policy']['Arn']
    )

except iam_client.exceptions.EntityAlreadyExistsException:
    print(f"IAM role {lambda_role_name} already exists. Getting its ARN...")
    response = iam_client.get_role(RoleName=lambda_role_name)
    lambda_role_arn = response['Role']['Arn']
    


In [ ]:
from io import BytesIO
import zipfile
import boto3

lambda_client = boto3.client('lambda',region_name=region)
lambda_function_name = f"{agent_name}-lambda"

s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda.py")
z.close()
zip_content = s.getvalue()

try:
    # Create Lambda Function
    lambda_function = lambda_client.create_function(
        FunctionName=lambda_function_name,
        Runtime='python3.12',
        Timeout=60,
        Role=lambda_role_arn,
        Code={'ZipFile': zip_content},
        Handler='lambda.lambda_handler'
    )
except lambda_client.exceptions.ResourceConflictException:
    print("Lambda function already exists, retrieving it")
    lambda_function = lambda_client.get_function(
        FunctionName=lambda_function_name
    )
    lambda_function = lambda_function['Configuration']


In [ ]:
agent_functions = [
    {
        'name': 'get_booking_details',
        'description': 'Retrieve details of a restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to retrieve",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_booking',
        'description': 'Create a new restaurant booking',
        'parameters': {
            "date": {
                "description": "The date of the booking in the format YYYY-MM-DD",
                "required": True,
                "type": "string"
            },
            "name": {
                "description": "Name to idenfity your reservation",
                "required": True,
                "type": "string"
            },
            "hour": {
                "description": "The hour of the booking in the format HH:MM",
                "required": True,
                "type": "string"
            },
            "num_guests": {
                "description": "The number of guests for the booking",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'delete_booking',
        'description': 'Delete an existing restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to delete",
                "required": True,
                "type": "string"
            }
        }
    },
]

In [ ]:
agent_action_group_description = """
Actions for getting table booking information, create a new booking or delete an existing booking"""

agent_action_group_name = "TableBookingsActionGroup"

agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [ ]:
# add permission to bedrock to invoke the lambda function in the action group
    
agent_arn = bedrock_agent_client.get_agent(agentId=agent_id)['agent']['agentArn']
print(f"Agent ARN: {agent_arn}")

try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function['FunctionArn'],
        StatementId="bedrock-invoke",
        Action="lambda:InvokeFunction",
        Principal="bedrock.amazonaws.com",
        SourceArn=agent_arn
    )
    print("Permission added successfully.")
    print("Response: {}".format(response))
except lambda_client.exceptions.ResourceConflictException as er:
    print(f"{er}\nremoving perminssion and recreating it")
    # note that if have deleted the agent and recreated it, the permission has also to be recreated for the new agent otheriwse you get a permssion error in invoke_agent()
    lambda_client.remove_permission(
        FunctionName=lambda_function['FunctionArn'],
        StatementId="bedrock-invoke"
    )
    response = lambda_client.add_permission(
        FunctionName=lambda_function['FunctionArn'],
        StatementId="bedrock-invoke",
        Action="lambda:InvokeFunction",
        Principal="bedrock.amazonaws.com",
        SourceArn=agent_arn
    )
    print("Permission added successfully.")
    print("Response: {}".format(response))
except Exception as e:
    print(f"Error adding permission: {str(e)}")


In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

In [ ]:
import logging
import pprint

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

def invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False, session_state=None):
    end_session: bool = False
    if not session_state:
        session_state = {}

    print(f"Query: {query}")
    print(f"Session ID: {session_id}")
    print(f"Agent ID: {agent_id}") 
    print(f"Alias ID: {alias_id}")
    print(f"Enable trace: {enable_trace}")
    print(f"Session state: {session_state}")
    print(f"End session: {end_session}")
    print("Invoking agent...")
    # invoke the agent API
    agent_response = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=enable_trace,
        endSession=end_session,
        sessionState=session_state
    )

    if enable_trace:
        logger.info(pprint.pprint(agent_response))

    event_stream = agent_response['completion']
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2, default=str))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [ ]:
alias_id = 'TSTALIASID'

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
query = "Hi, my name is Tim, I want to create a booking for 2 people, at 8pm on the 5th of May 2024."
#query = "Hi, my name is Yannis, I want to create a booking at 8pm on the 5th of May 2024."
print(session_id)

In [ ]:
#query = "for 5 people"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)

print(response)